In [15]:
import pandas as pd

# Read the CSV file
gpa_data = pd.read_csv('gpa.csv')

# Exclude courses from the year 2024
gpa_data = gpa_data[gpa_data['Year'] != 2024]

# Group by 'Course Title' and 'Year', then check if all terms are present
course_year_group = gpa_data.groupby(['Course Title', 'Year'])['Term'].nunique().reset_index()

# Filter the courses that have all four terms
courses_all_terms = course_year_group[course_year_group['Term'] == 4]

# Merge the filtered courses back with the original data to keep all columns
filtered_gpa_data = gpa_data.merge(courses_all_terms[['Course Title', 'Year']], on=['Course Title', 'Year'], how='inner')

filtered_gpa_data

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C+,C,C-,D+,D,D-,F,W,Primary Instructor,Students
0,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,LAB,0,19,3,...,0,0,1,0,0,0,3,0,"Allen, Betty J",29
1,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,16,2,...,0,0,0,0,0,0,0,0,"Owens, Nikolas C",23
2,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,1,15,5,...,0,0,0,0,1,0,1,0,"Condon, Elizabeth G",24
3,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,0,18,3,...,0,0,2,1,0,0,0,0,"Segale, Magdalyn L",25
4,2023,Fall,2023-fa,DANC,100,Intro to Contemporary Dance,ONL,17,5,1,...,0,0,0,0,0,0,0,0,"Gonzalez, Gabriel B",25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,2015,Winter,2015-wi,BADM,350,IT for Networked Organizations,ONL,2,8,2,...,1,2,2,1,0,0,2,0,"Sachdev, Vishal",28
2912,2015,Winter,2015-wi,BADM,380,International Business,ONL,0,9,12,...,0,2,0,3,0,0,0,0,"Bucheli, Marcelo",42
2913,2015,Winter,2015-wi,ECON,102,Microeconomic Principles,ONL,10,71,9,...,3,0,0,0,0,0,2,0,"Vazquez, Jose J",104
2914,2015,Winter,2015-wi,ECON,203,Economic Statistics II,ONL,10,6,9,...,7,5,0,2,2,0,5,1,"Petry, Joseph A",57


In [17]:
# Accumulate Spring & Summer / Fall & Winter for each course to represent the european semesters

# Create a new column for European semester
def map_to_european_semester(term):
    if term in ['Spring', 'Summer']:
        return 'SoSe'
    elif term in ['Fall', 'Winter']:
        return 'WiSe'
    else:
        return term

filtered_gpa_data['European Semester'] = filtered_gpa_data['Term'].apply(map_to_european_semester)
grade_columns = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F', 'W']

# Group by the new semester column and the primary key columns
european_semester = filtered_gpa_data.groupby(['Course Title', 'Year', 'Number', 'Subject', 'European Semester']).agg({
    'Students': 'sum',
    **{grade: 'sum' for grade in grade_columns}
}).reset_index()

european_semester.to_csv('european_semester.csv', index=False)

european_semester


,Course Title,Year,Number,Subject,European Semester,Students,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W
0,Accounting and Accountancy II,2021,202,ACCY,SoSe,506,37,84,37,84,124,41,30,29,29,0,10,0,1,3
1,Accounting and Accountancy II,2021,202,ACCY,WiSe,296,37,62,16,50,63,19,23,16,6,0,1,0,3,0
2,Advanced Financial Reporting,2021,410,ACCY,SoSe,218,30,48,16,22,47,14,9,19,3,3,5,1,1,0
3,Advanced Financial Reporting,2021,410,ACCY,WiSe,179,26,55,23,18,25,17,3,4,2,0,3,0,3,2
4,Advanced Financial Reporting,2021,506,ACCY,WiSe,67,8,20,8,11,4,7,4,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,World Animal Resources,2021,205,ANSC,WiSe,392,174,114,36,12,18,12,1,6,2,0,3,4,10,0
390,World Religions,2021,110,REL,SoSe,526,191,140,55,27,32,8,15,11,8,5,3,4,27,1
391,World Religions,2021,110,REL,WiSe,511,280,93,43,22,24,13,12,6,3,2,3,1,9,3
392,World Religions,2022,110,REL,SoSe,640,368,98,52,23,24,16,8,13,4,2,4,4,24,4


In [18]:
# Define a dictionary to map letter grades to numeric values
grade_mapping = {'A+': 1.0, 'A': 1.0, 'A-': 1.3, 'B+': 1.7, 'B': 2.0, 'B-': 2.3, 'C+': 2.7, 'C': 3.0, 'C-': 3.3, 'D+': 3.7, 'D':3.7, 'D-': 4.0, 'F': 5.0}

# Calculate the total numeric grade for each row
european_semester['Average Numeric Grade'] = (
    (filtered_gpa_data['A+'] * grade_mapping['A+'] +
    filtered_gpa_data['A'] * grade_mapping['A'] +
    filtered_gpa_data['A-'] * grade_mapping['A-'] +
    filtered_gpa_data['B+'] * grade_mapping['B+'] +
    filtered_gpa_data['B'] * grade_mapping['B'] +
    filtered_gpa_data['B-'] * grade_mapping['B-'] +
    filtered_gpa_data['C+'] * grade_mapping['C+'] +
    filtered_gpa_data['C'] * grade_mapping['C'] +
    filtered_gpa_data['C-'] * grade_mapping['C-'] +
    filtered_gpa_data['D+'] * grade_mapping['D+'] +
    filtered_gpa_data['D'] * grade_mapping['D'] +
    filtered_gpa_data['D-'] * grade_mapping['D-'] +
    filtered_gpa_data['F'] * grade_mapping['F'] )
    / filtered_gpa_data[['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']].sum(axis=1)
)


# Calculate the average numeric grade for each row

european_semester.to_csv('european_semester_avg.csv', index=False)

european_semester




,Course Title,Year,Number,Subject,European Semester,Students,A+,A,A-,B+,...,B-,C+,C,C-,D+,D,D-,F,W,Average Numeric Grade
0,Accounting and Accountancy II,2021,202,ACCY,SoSe,506,37,84,37,84,...,41,30,29,29,0,10,0,1,3,1.596552
1,Accounting and Accountancy II,2021,202,ACCY,WiSe,296,37,62,16,50,...,19,23,16,6,0,1,0,3,0,1.217391
2,Advanced Financial Reporting,2021,410,ACCY,SoSe,218,30,48,16,22,...,14,9,19,3,3,5,1,1,0,1.383333
3,Advanced Financial Reporting,2021,410,ACCY,WiSe,179,26,55,23,18,...,17,3,4,2,0,3,0,3,2,1.368000
4,Advanced Financial Reporting,2021,506,ACCY,WiSe,67,8,20,8,11,...,7,4,3,0,0,2,0,0,0,1.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,World Animal Resources,2021,205,ANSC,WiSe,392,174,114,36,12,...,12,1,6,2,0,3,4,10,0,1.739130
390,World Religions,2021,110,REL,SoSe,526,191,140,55,27,...,8,15,11,8,5,3,4,27,1,2.340000
391,World Religions,2021,110,REL,WiSe,511,280,93,43,22,...,13,12,6,3,2,3,1,9,3,2.059574
392,World Religions,2022,110,REL,SoSe,640,368,98,52,23,...,16,8,13,4,2,4,4,24,4,1.985106
